CNN

![network](./image/CNN-1.png)

1. CNNの構造

    畳み込み層（Convolutional Layer）、活性化関数（ReLUなど）、プーリング層（Pooling Layer）、全結合層（Fully Connected Layer）から主に構成されています

    1.1 畳み込み層（Convolutional Layer）
    
    目的：入力画像から特徴を抽出する。
    
    動作：小さなフィルタ（カーネル）を入力データ上でスライドさせ、フィルタと入力データとの畳み込みを計算します。このプロセスにより、エッジやテクスチャなどの局所的特徴が抽出されます。

    1.2 活性化関数（Activation Function）
        
    目的：非線形性を導入する。
    
    一般的な選択：ReLU（Rectified Linear Unit）が一般的に使用されます。これは、ネットワークがより複雑な関数を学習できるようにするためです。

    1.3 プーリング層（Pooling Layer）
        
    目的：特徴マップの次元を削減し、計算コストを減らすとともに、特徴の位置変動に対する感度を下げる。
    
    動作：通常、最大プーリング（Max Pooling）または平均プーリング（Average Pooling）を使用して、特定の領域（例えば2x2ピクセル）の最大値または平均値を取り、出力サイズを減少させます。

    1.4 全結合層（Fully Connected Layer）
        
    目的：抽出された特徴を基に、最終的な分類または回帰タスクを行う。
    
    動作：ネットワークのこの部分では、以前の層からの出力を受け取り、特定のクラスに属する確率などの予測を行います。

    1.5 そのほか
    
    正規化層（Normalization Layer）：学習プロセスを安定させ、過学習を防ぐために使用されることがあります。
    
    ドロップアウト層（Dropout Layer）：過学習を防ぐために、学習中にネットワークの一部の接続をランダムに切断します。

2. Text-CNN

    2.1 論文　[リンク](https://aclanthology.org/D14-1181.pdf)

    2.2 ストラクチャー

    ![structure](./image/textCNN.png)
    
    [source code](../../source_code/deep_learning/text-CNN.py)

3. その他

    [TextCNNによる日本語評判文書分類](https://qiita.com/donaldchi/items/321d96ced569d6b8389d)
    
    [畳み込みニューラルネットワーク（CNN）とは？3層の仕組み・ディープラーニング他手法との関係を徹底解説！](https://ai-market.jp/purpose/cnn/#:~:text=CNN%E3%81%AF%E4%BA%BA%E9%96%93%E3%81%AE%E8%84%B3,%E6%A7%8B%E9%80%A0%E3%82%92%E3%81%97%E3%81%A6%E3%81%84%E3%81%BE%E3%81%99%E3%80%82)


In [ ]:


import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Embedding(vocab_size, embedding_size)
        self.Weight = nn.Linear(self.num_filters_total, num_classes, bias=False)
        self.Bias = nn.Parameter(torch.ones([num_classes]))
        self.filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (size, embedding_size)) for size in filter_sizes])

    def forward(self, X):
        embedded_chars = self.W(X) # [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]

        pooled_outputs = []
        for i, conv in enumerate(self.filter_list):
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            h = F.relu(conv(embedded_chars))
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_sizes[i] + 1, 1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]
        model = self.Weight(h_pool_flat) + self.Bias # [batch_size, num_classes]
        return model

if __name__ == '__main__':
    embedding_size = 2 # embedding size
    sequence_length = 3 # sequence length
    num_classes = 2 # number of classes
    filter_sizes = [2, 2, 2] # n-gram windows
    num_filters = 3 # number of filters

    # 3 words sentences (=sequence_length is 3)
    sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
    labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    vocab_size = len(word_dict)

    model = TextCNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
    targets = torch.LongTensor([out for out in labels]) # To using Torch Softmax Loss function

    # Training
    for epoch in range(5000):
        optimizer.zero_grad()
        output = model(inputs)

        # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
        loss = criterion(output, targets)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Test
    test_text = 'sorry hate you'
    tests = [np.asarray([word_dict[n] for n in test_text.split()])]
    test_batch = torch.LongTensor(tests)

    # Predict
    predict = model(test_batch).data.max(1, keepdim=True)[1]
    if predict[0][0] == 0:
        print(test_text,"is Bad Mean...")
    else:
        print(test_text,"is Good Mean!!")